In [6]:
from __future__ import print_function
import cv2
import numpy as np
from skimage.metrics import structural_similarity
import matplotlib.pyplot as plt
from cv2 import cv2 as cv2
from PIL import Image
import imagehash
from moviepy.editor import VideoFileClip
import os
from collections import deque

## parameter

In [42]:
gap_factor=10 ## num of detection within each second
match_ratio=0.35 ##the distance ratio thershold between best match and second match
good_match_rate=0.7 ##the rate of matches to be chosen in all
ransacReprojThreshold=3
min_match_count=5


## anotation

for keypoint, attribute to use are:
    pt 
    angle
    response
    size
    octave
    class_id：object
for match, attribute to use are:
    distance
    queryIdx
    trainIdx
    imgIdx
    
    print(kp1[i.queryIdx].pt) location before move
    print(kp2[i.trainIdx].pt) location after move

## test begin

In [14]:
def main():
    
    ##load test video
    video=os.path.abspath(os.curdir)+'/down.mp4'
    videoCapture = cv2.VideoCapture(video)
    
    ##detect for gap_factor #gap_dected frames
    fps = videoCapture.get(cv2.CAP_PROP_FPS)
    print(fps)
    gap_dected=int(fps//gap_factor)
    
    ##load first frame
    success, frame_old = videoCapture.read()
    i,j=0,0
    _SaveImage(frame_old[80:frame_old.shape[0]-70,:],os.path.abspath(os.curdir)+'/test',j)
    print('save image:',i)
    
#     predict=deque([],maxlen=gap_factor)
#     for _ in range(gap_factor):
#         predict.append(None)
    
    while success:
        i+=1
    ##finish process if all frames loaded
        success, frame_new = videoCapture.read()
        if success==False:
            print('vedio detection finish')
#           sys.exit() 
            return 
        if (i % gap_dected == 0):
            j+=1
            _SaveImage(frame_new[80:frame_new.shape[0]-70,:],os.path.abspath(os.curdir)+'/test',j)
            print('save image:',i)
            
    ##predict 
            if _AlignImages(frame_old,frame_new) != None:
                predict=_AlignImages(frame_old,frame_new)
                
                res=_Predict(predict)
                print(i//30,'s',i%30,'frame',res)
            
            
        frame_old=frame_new
    
            
        
        

        
        
    
    

In [68]:
def _SaveImage(image,addr,num):
    address = addr +'/'+ str(num)+ '.jpg'
    cv2.imwrite(address,image)

def _AlignImages(img1,img2):
    # initiate ORB detector
    orb = cv2.ORB_create(5000)
    
    ##check bilateral banner, eliminate stick part
#     up_idx,down_idx=_SplitImage(img1,img2)
#     img1 = img1[up_idx:down_idx,:]
#     img2= img2[up_dx:down_idx,:]
    img1 = img1[80:img1.shape[0]-70,:]
    img2 = img2[80:img2.shape[0]-70,:]
    img_2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    count=0
    for i in img_2.sum(axis=1):
        if i==0:
            count+=1
    if count/img_2.sum(axis=1).shape[0]>0.2:
        return 'open','picture'
    
    ##find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)
    if des2 is None: 
        return 'click','in'
    
    ##match with Brute-Force
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    ##find best 2 matches for each kp
    matches = bf.knnMatch(des1, trainDescriptors = des2, k = 2)
#     print(len(matches))
    ##ratio test
    good = [m for (m,n) in matches if m.distance < match_ratio*n.distance]
    ##sort and choose the best part of matches
#     good = sorted(good, key = lambda x:x.distance)
#     good = good[:int(len(good) * good_match_rate)]
    
    if len(good)>min_match_count:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        ##find (3x3) transform matrix
        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,ransacReprojThreshold)
        t_x = M[0][2]
        t_y = M[1][2]
        return (t_x,t_y)
    else:
        return None
        
        

def _Predict(predict):
    
    res= __Pattern(predict[0],predict[1])
    return res
    
def __Pattern(t_x,t_y):
    if type(t_x)!=str and type(t_y)!=str:
        res=[]
        if abs(t_x)<1 and abs(t_y)<1:
            res.append('stop')
        if abs(t_y/t_x)>2:
            if (t_y>5):
                res.append('down')
            if (t_y<-1):
                res.append('up')
        
        if abs(t_x/t_y)>5:
            if (t_x>3):
                res.append('right')
            if(t_x<-3):
                res.append('left')
        if len(res)==0:
            return 'stop'
        else:
            major={'up':0,'right':0,'left':0,'stop':0}
            for e in res:
                major[e]+=1
            for k, v in major.items():
                if v == max(major.values()):
                    return k        
        
    else:
#         if t_x=='click' and t_y=='in':
        return t_x+t_y
        
        
        
        
        
#         if abs(t_x)<0.1 and abs(t_y)<0.1:
#     #         print('stop')
#             return 'stop'
    
#         elif abs(t_y/t_x)>2:
#     #         print('up,down')
#             if (t_y>5):
#     #             print('down')
#                 return 'down'
# #             elif (t_y<-0.5):
# #     #             print('up')
# #                 return 'up'
#             else:
#                 return 'up'
            
#         elif abs(t_x/t_y)>5:
#             if (t_x>3):
#     #             print('down')
#                 return 'right'
#             elif(t_x<-3):
#                 return 'left'
#             else:
#     #             print('up')
#                 return 'stop'


#         else:
#             return 'stop'
   
        
        
          
    
    
    
    
    
    
    
    

In [69]:
# major={'stop':1,'up':0,'right':0,'left':0}

In [70]:
main()

25.972602739726028
save image: 0
save image: 2
0 s 2 frame stop
save image: 4
0 s 4 frame stop
save image: 6
0 s 6 frame stop
save image: 8
0 s 8 frame stop
save image: 10
0 s 10 frame stop
save image: 12
0 s 12 frame stop
save image: 14
0 s 14 frame stop
save image: 16
0 s 16 frame stop
save image: 18
0 s 18 frame stop
save image: 20
0 s 20 frame stop
save image: 22
0 s 22 frame stop
save image: 24
0 s 24 frame up
save image: 26
0 s 26 frame up
save image: 28
0 s 28 frame up
save image: 30
1 s 0 frame up
save image: 32
1 s 2 frame up
save image: 34
1 s 4 frame stop
save image: 36
1 s 6 frame stop
save image: 38
1 s 8 frame stop
save image: 40
1 s 10 frame stop
save image: 42
1 s 12 frame stop
save image: 44
1 s 14 frame stop
save image: 46
1 s 16 frame up
save image: 48
1 s 18 frame up
save image: 50
1 s 20 frame up
save image: 52
1 s 22 frame up
save image: 54
1 s 24 frame up
save image: 56
1 s 26 frame stop
save image: 58
1 s 28 frame stop
save image: 60
2 s 0 frame stop
save image:

In [11]:
a=deque([],maxlen=2)


In [12]:
a.append(1)
a.append(None)

In [13]:
a


deque([1, None])

In [20]:
test()

'stop'

In [82]:
a=1

In [83]:
type(a)

int

In [84]:
type(str(a))

str

In [85]:
type(a)!=str

True